In [ ]:
import numpy as np
import pandas as pd

import ast
import random
import lmstudio as lms

from IPython.display import Markdown

In [ ]:
raw_link = pd.read_csv('postprocess.csv')
raw_tthc = pd.read_csv('data/raw_tthc.csv', dtype={'maThuTuc': str})

In [ ]:
cols = ['link', 'Mã thủ tục hành chính liên quan', 'Số quyết định', 'Tên thủ tục', 'Cấp thực hiện', 'Loại thủ tục', 'Lĩnh vực', 
        'Trình tự thực hiện', 'Cách thức thực hiện', 'Thành phần hồ sơ', 'Đối tượng thực hiện', 'Cơ quan thực hiện', 
        'Cơ quan có thẩm quyền', 'Địa chỉ tiếp nhận hồ sơ', 'Cơ quan được ủy quyền', 'Cơ quan phối hợp', 'Kết quả thực hiện', 
        'Căn cứ pháp lý', 'Yêu cầu, điều kiện thực hiện', 'Từ khóa', 'Mô tả']

In [ ]:
raw_tthc.columns = cols

In [ ]:
pattern = []

pattern.append("Bạn đang cố gắng xác định xem câu trả lời có hiểu sai ngữ cảnh và mục đích của câu hỏi hay không.")
pattern.append("Bạn đang cố gắng xác định xem câu trả lời có sự mâu thuẫn về mặt thực tế so với kiến thức liên quan hay không. Một số thông tin trong câu trả lời có thể đã được bịa ra.")
pattern.append("Bạn đang cố gắng xác định xem câu trả lời có quá chung chung hoặc quá chi tiết so với mức độ cụ thể phù hợp để trả lời câu hỏi hay không.")
pattern.append("Bạn đang cố gắng xác định xem câu trả lời có thể được suy ra đúng từ kiến thức liên quan hay không.")

In [ ]:
config = {"temperature": 0.7, "maxTokens": 512, "topPSampling": 0.9}

In [ ]:
evaluate_template = \
"""Bạn sẽ đóng vai trò là một người đánh giá câu trả lời (answer judge). Với một câu hỏi và câu trả lời, mục tiêu của bạn là xác định xem câu trả lời được cung cấp có chứa thông tin không đúng sự thật hoặc thông tin ảo giác (hallucinated information) hay không. {pattern}

#Câu hỏi#: {question}

#Câu trả lời#: {answer}

#Đánh giá của bạn#:

Bạn nên cố gắng hết sức để xác định xem câu trả lời có chứa thông tin không đúng sự thật hoặc thông tin ảo giác hay không. Câu trả lời bạn đưa ra PHẢI là "Có" hoặc "Không".
"""

In [8]:
model = lms.llm("llama-3.2-1b-instruct")

df = pd.DataFrame(columns=['link', 'cauTraLoiAoGiac', 'pattern', 'danhGia'])

for i in range(len(raw_link)):
    p = raw_link['pattern'][i]
    question = raw_link['cauHoi'][i]
    answer = raw_link['cauTraLoiAoGiac'][i]
    prompt = evaluate_template.format(pattern=pattern[p], question=question, answer=answer)

    try:
        completion = model.respond(prompt, config=config)
        output = str(completion)
    except Exception as e:
        output = ""
        print(e)
        break

    results = [raw_link['link'][i], answer, p, output]
    df.loc[i, ['link', 'cauTraLoiAoGiac', 'pattern', 'danhGia']] = results    

    print(f'Done of {len(raw_link)}: {i+1}', end='\r')

In [9]:
df.to_csv("data/llama-3.2-1b-instruct.csv")

In [10]:
model = lms.llm("gemma-3-1b-it")

df = pd.DataFrame(columns=['link', 'cauTraLoiAoGiac', 'pattern', 'danhGia'])

for i in range(len(raw_link)):
    p = raw_link['pattern'][i]
    question = raw_link['cauHoi'][i]
    answer = raw_link['cauTraLoiAoGiac'][i]
    prompt = evaluate_template.format(pattern=pattern[p], question=question, answer=answer)

    try:
        completion = model.respond(prompt, config=config)
        output = str(completion)
    except Exception as e:
        output = ""
        print(e)
        break

    results = [raw_link['link'][i], answer, p, output]
    df.loc[i, ['link', 'cauTraLoiAoGiac', 'pattern', 'danhGia']] = results    

    print(f'Done of {len(raw_link)}: {i+1}', end='\r')

In [11]:
df.to_csv("data/gemma-3-1b-it.csv")

In [12]:
model = lms.llm("llama-3.2-1b-instruct")
result = model.respond("What is the meaning of life?", config={"temperature": 0.7, "maxTokens": 512, "topPSampling": 0.9})

Markdown(str(result))


The question of the meaning of life has been a profound and intriguing topic that philosophers, theologians, scientists, and everyday people have grappled with for centuries. There is no one definitive answer, as it depends on individual perspectives, cultural background, personal experiences, and philosophical outlooks.

Here are some possible interpretations:

1. **Biological perspective**: From a biological standpoint, the meaning of life might be to survive, reproduce, and perpetuate one's genes. This could lead to an evolutionary focus on basic needs like food, shelter, and reproduction.
2. **Philosophical perspective**: Many philosophers have proposed various definitions of the meaning of life, such as:
	* Purpose: To find purpose or significance in life.
	* Happiness: To seek happiness or fulfillment.
	* Fulfillment: To achieve personal growth and self-actualization.
	* Connection: To form meaningful relationships with others.
3. **Religious perspective**: For many people of faith, the meaning of life is to fulfill a divine plan or purpose, often tied to their specific religion's teachings.
4. **Spiritual perspective**: Some spiritual traditions believe that the meaning of life is to connect with a higher power, achieve enlightenment, or experience transcendence.
5. **Existentialist perspective**: Existentialists like Jean-Paul Sartre and Albert Camus argue that individuals must create their own meaning in life, as there is no inherent or universal purpose.
6. **Humanistic perspective**: Humanists emphasize the importance of human values, such as empathy, compassion, and kindness, to create a meaningful life.
7. **Personal growth perspective**: Many people believe that the meaning of life is to learn, grow, and improve themselves, both personally and professionally.

Ultimately, the meaning of life is subjective and personal, influenced by factors like culture, upbringing, experiences, and individual values.

What do you think? What gives your life meaning?

In [13]:
model = lms.llm("gemma-3-1b-it")
result = model.respond("What is the meaning of life?")

Markdown(str(result))


Okay, this is the big one! The question "What is the meaning of life?" has plagued philosophers and individuals for millennia. There's no single, universally accepted answer – it’s a deeply personal and often evolving quest. Here's a breakdown of different perspectives and approaches to thinking about it:

**1. Philosophical Perspectives:**

* **Nihilism:** This view argues that life is inherently without objective meaning, purpose, or intrinsic value. It can be unsettling, but some find freedom in accepting this – no pre-ordained reason for being.
* **Existentialism:**  This philosophy emphasizes individual freedom and responsibility. Existentialists believe we are born into a meaningless universe, and it's *our* task to create our own meaning through our choices and actions. Key figures include Sartre and Camus.
    * **Sartre’s idea of ‘bad faith’** suggests that people try to avoid the burden of creating their own meaning by conforming to societal expectations or pre-determined roles.
* **Absurdism:** Closely related to existentialism, absurdism recognizes the fundamental conflict between humanity's innate desire for meaning and the meaningless universe. It advocates embracing this absurdity rather than trying to resolve it. Camus’ *The Myth of Sisyphus* is a key text on this.
* **Hedonism:**  This suggests that pleasure and happiness are the ultimate goals in life. The meaning lies in maximizing enjoyment and minimizing suffering. However, many hedonists criticize it for being shallow.
* **Utilitarianism:** This ethical theory proposes that the best action is the one that maximizes overall happiness and well-being for the greatest number of people.  Meaning comes from contributing to a better world.


**2. Psychological Perspectives:**

* **Purpose & Meaning as a Construct:** Psychology increasingly suggests that we *create* meaning in our lives rather than discovering it. This isn't about finding a grand, overarching purpose; it’s about identifying goals, values, and connections that give your life significance.
* **Self-Actualization (Maslow):**  This theory proposes that humans are driven to fulfill their potential – to become the best version of themselves. Meaning comes from pursuing growth, creativity, and a sense of accomplishment.
* **Positive Psychology:** Focuses on cultivating happiness, resilience, and well-being through practices like gratitude, mindfulness, and strong social connections.

**3. Religious & Spiritual Perspectives:**

* **Purpose Driven by God/Higher Power:** Many religions offer answers about the meaning of life – often involving serving a divine being, following religious doctrines, and striving for spiritual enlightenment or union with the divine.
* **Karma & Reincarnation (Hinduism, Buddhism):** These belief systems suggest that our actions have consequences that shape our future lives, ultimately contributing to a cycle of rebirth. The meaning is found in ethical conduct and liberation from suffering. 


**4.  More Personal Perspectives:**

Ultimately, the "meaning of life" isn't a universal truth waiting to be discovered. It’s often something *you* define for yourself. Here are some common sources of personal meaning:

* **Relationships:** Connecting with loved ones and building meaningful relationships.
* **Contribution:** Making a difference in the world through work, volunteering, or acts of kindness.
* **Creativity & Self-Expression:** Pursuing artistic endeavors, hobbies, or simply expressing yourself authentically.
* **Learning & Growth:**  Continuously expanding your knowledge and skills.
* **Experiences:**  Seeking out new adventures and creating lasting memories.
* **Simply Enjoying the Moment:** Appreciating the beauty of everyday life.


**Important Considerations:**

* **It’s Okay to Not Know:**  Don't feel pressured to find a definitive answer right away. It’s a journey, not a destination.
* **Meaning Can Evolve:** Your understanding of meaning can change over time as you grow and experience new things.
* **Small Moments Matter:** Even seemingly insignificant acts of kindness or appreciation can contribute to a sense of purpose.

**Resources for Further Exploration:**

* **Stanford Encyclopedia of Philosophy:** [https://plato.stanford.edu/entries/philosophy-meaning-life/](https://plato.stanford.edu/entries/philosophy-meaning-life/)
* **Psychology Today - What is the meaning of life?** [https://www.psychologytoday.com/us/blog/you-are-not-alone/201812/what-is-the-meaning-of-life](https://www.psychologytoday.com/us/blog/you-are-not-alone/201812/what-is-the-meaning-of-life)
* **Book: "Man's Search for Meaning" by Viktor Frankl:**  A powerful exploration of finding meaning in suffering.


To help me give you an even more tailored response, could you tell me:

*   **What kind of perspective are you interested in exploring?** (e.g., philosophical, spiritual, psychological)
*   **What's important to you in life right now?** (e.g., relationships, creativity, making a difference, personal growth?)

In [15]:
model = lms.llm("llama-3.2-1b-instruct")

df = pd.DataFrame(columns=['link', 'cauTraLoi', 'danhGia'])

for i in range(len(raw_link)):
    question = raw_link['cauHoi'][i]
    answer = raw_link['cauTraLoi'][i]
    prompt = evaluate_template.format(pattern="", question=question, answer=answer)

    try:
        completion = model.respond(prompt, config=config)
        output = str(completion)
    except Exception as e:
        output = ""
        print(e)
        break

    results = [raw_link['link'][i], answer, output]
    df.loc[i, ['link', 'cauTraLoi', 'danhGia']] = results    

    print(f'Done of {len(raw_link)}: {i+1}', end='\r')

In [16]:
df.to_csv("data/llama-3.2-1b-instruct_right.csv")

In [17]:
model = lms.llm("gemma-3-1b-it")

df = pd.DataFrame(columns=['link', 'cauTraLoi', 'danhGia'])

for i in range(len(raw_link)):
    question = raw_link['cauHoi'][i]
    answer = raw_link['cauTraLoi'][i]
    prompt = evaluate_template.format(pattern="", question=question, answer=answer)

    try:
        completion = model.respond(prompt, config=config)
        output = str(completion)
    except Exception as e:
        output = ""
        print(e)
        break

    results = [raw_link['link'][i], answer, output]
    df.loc[i, ['link', 'cauTraLoi', 'danhGia']] = results    

    print(f'Done of {len(raw_link)}: {i+1}', end='\r')

In [18]:
df.to_csv("data/gemma-3-1b-it_right.csv")